using an example from: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maggi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

C:\Users\maggi\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
# spacy for lemmatization
import spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Logging
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [8]:
#All Gutenberg metadata
metadata = pd.read_csv('../data/metadata/metadata.csv')

In [9]:
#Select detective/mystery fiction and tales in English
kw = ['fiction','tale']
subj = ['detective', 'mystery','mysteries']
metadata = metadata[metadata['language']=='[\'en\']']
metadata = metadata[metadata['subjects'].apply(lambda x: any([k in x.lower() for k in kw]))]
metadata = metadata[metadata['subjects'].apply(lambda x: any([k in x.lower() for k in subj]))]

#Ids to list
ids = list(metadata['id'])

#List the ids and path for these titles
txts = [[id, '../data/text\\' + id + '_text.txt'] for id in ids]

#note any files that don't exist (duplicates or files that were not utf8) and remove them from the metadata df
docs = []

for idx, filename in tqdm(txts):
    try:
        with open(filename, 'r', encoding = 'utf8') as txt:
            # added encoding = 'utf8' to prevent a continuation byte error
            #txt = f
            #pdf = pdftotext.PDF(f)
            txt = '\n\n'.join(txt).lower()
            txt = re.sub('-\s+', '', txt)
            txt = re.sub('--',' ', txt)
            txt = re.sub("\'", "", txt)
            txt = ' '.join(re.findall('[a-z\-]{2,}', txt))
            txt = txt[0:20000]
            docs.append(txt)
    except FileNotFoundError:
        print('WARNING: file not found ', filename)
        metadata = metadata[metadata['id']!=idx]

#Does everything match?
print(len(metadata) == len(docs))
print(len(metadata))

  0%|          | 0/1093 [00:00<?, ?it/s]

True
1052


In [10]:
docs[0]

'proofreaders the secret of the tower by anthony hope author of the prisoner of zenda rupert of hentzau etc contents doctor marys paying guest ii the general remembers iii mr saffron at home iv professional etiquette familiar implement vi odd story of captain duggle vii gentlemanly stranger viii captain alec raises his voice ix doctor marys ultimatum that magical word morocco xi the car behind the trees xii the secret of the tower xiii right of conquest xiv the scepter in the grave xv normal case xvi dead majesty xvii the chief mourners xviii the gold and the treasure chapter doctor marys paying guest just in time wasnt it asked mary arkroyd two days before the the ceremony mercifully it had all been kept very quiet because it was only three months since poor gilly was killed forget whether you ever met gilly my half-brother you know only once in collingham gardens he had an exeat and dashed in one saturday morning when we were just finishing our work dont you remember yes think do but

In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(docs))

#print(data_words[:1])

In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['proofreaders', 'the', 'secret', 'of', 'the', 'tower', 'by', 'anthony', 'hope', 'author', 'of', 'the', 'prisoner', 'of', 'zenda', 'rupert', 'of', 'hentzau', 'etc', 'contents', 'doctor', 'marys', 'paying_guest', 'ii', 'the', 'general', 'remembers', 'iii', 'mr', 'saffron', 'at', 'home', 'iv', 'professional', 'etiquette', 'familiar', 'implement', 'vi', 'odd', 'story', 'of', 'captain', 'duggle', 'vii', 'gentlemanly', 'stranger', 'viii', 'captain', 'alec', 'raises', 'his', 'voice', 'ix', 'doctor', 'marys', 'ultimatum', 'that', 'magical', 'word', 'morocco', 'xi', 'the', 'car', 'behind', 'the', 'trees', 'xii', 'the', 'secret', 'of', 'the', 'tower', 'xiii', 'right', 'of', 'conquest', 'xiv', 'the', 'scepter', 'in', 'the', 'grave', 'xv', 'normal', 'case', 'xvi', 'dead', 'majesty', 'xvii', 'the', 'chief', 'mourners', 'xviii', 'the', 'gold', 'and', 'the', 'treasure', 'chapter', 'doctor', 'marys', 'paying_guest', 'just', 'in', 'time', 'wasnt', 'it', 'asked', 'mary_arkroyd', 'two', 'days', 'before'

In [13]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
#nlp = spacy.load('en', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['proofreader', 'secret', 'tower', 'anthony', 'hope', 'author', 'prisoner', 'zenda', 'rupert', 'hentzau', 'content', 'doctor', 'mary', 'paying_gu', 'ii', 'general', 'remember', 'saffron', 'home', 'professional', 'etiquette', 'familiar', 'implement', 'vi', 'odd', 'story', 'captain', 'duggle', 'vii', 'gentlemanly', 'strange', 'viii', 'captain', 'alec', 'raise', 'voice', 'ix', 'doctor', 'mary', 'ultimatum', 'magical', 'word', 'morocco', 'car', 'tree', 'xii', 'secret', 'tower', 'xiii', 'right', 'conquest', 'xiv', 'scepter', 'grave', 'xv', 'normal', 'case', 'xvi', 'dead', 'majesty', 'xvii', 'chief', 'mourner', 'xviii', 'gold', 'treasure', 'chapter', 'doctor', 'mary', 'paying_gu', 'time', 'be', 'ask', 'day', 'ceremony', 'mercifully', 'keep', 'quiet', 'month', 'poor', 'gilly', 'kill', 'forget', 'ever', 'meet', 'gilly', 'half', 'brother', 'know', 'collingham', 'garden', 'exeat', 'dash', 'saturday', 'morning', 'finishing', 'work', 'do', 'remember', 'think', 'engagement', 'd', 'go', 'color', 'c

In [15]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 3), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 6), (26, 1), (27, 3), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 5), (36, 1), (37, 1), (38, 1), (39, 3), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 2), (48, 1), (49, 1), (50, 4), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 5), (60, 3), (61, 1), (62, 6), (63, 3), (64, 1), (65, 1), (66, 1), (67, 1), (68, 8), (69, 2), (70, 8), (71, 4), (72, 1), (73, 2), (74, 1), (75, 1), (76, 3), (77, 1), (78, 3), (79, 1), (80, 1), (81, 3), (82, 1), (83, 2), (84, 1), (85, 2), (86, 3), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 1), (109, 2), (110, 4)

In [16]:
id2word[871]

'voice'

In [17]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('absolute', 1),
  ('accent', 1),
  ('accept', 1),
  ('accessible', 1),
  ('accustomed', 1),
  ('ache', 1),
  ('acquaint', 1),
  ('acre', 1),
  ('act', 1),
  ('active', 1),
  ('activity', 1),
  ('actually', 1),
  ('add', 3),
  ('address', 1),
  ('admiration', 1),
  ('admit', 1),
  ('advice', 1),
  ('affect', 1),
  ('afford', 1),
  ('afraid', 2),
  ('afternoon', 2),
  ('agent', 1),
  ('aggressive', 1),
  ('agreeable', 1),
  ('air', 1),
  ('alec', 6),
  ('alecs', 1),
  ('almost', 3),
  ('alone', 1),
  ('also', 2),
  ('always', 1),
  ('ambitious', 1),
  ('amenity', 1),
  ('amused', 1),
  ('angry', 1),
  ('anna', 5),
  ('answer', 1),
  ('anthony', 1),
  ('antiquity', 1),
  ('anyhow', 3),
  ('apparently', 2),
  ('appear', 1),
  ('approach', 1),
  ('ardent', 1),
  ('arkroyd', 1),
  ('arkroyds', 1),
  ('arm', 2),
  ('armistice', 2),
  ('arrangement', 1),
  ('arrival', 1),
  ('ask', 4),
  ('aspect', 1),
  ('assign', 1),
  ('assist', 1),
  ('astonish', 1),
  ('attentive', 1),
  ('attractive',

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
lda_model.save('../model_artifacts/model.lda')

In [24]:
model = gensim.models.ldamodel.LdaModel.load('../model_artifacts/model.lda')

In [25]:
model.print_topics()

[(0,
  '0.089*"mary" + 0.070*"leslie" + 0.038*"cynthia" + 0.035*"jeanne" + 0.008*"delia" + 0.005*"naylor" + 0.002*"beaumaroy" + 0.002*"saffron" + 0.002*"inkston" + 0.002*"alec"'),
 (1,
  '0.000*"portliness" + 0.000*"ranchero" + 0.000*"mina" + 0.000*"mozo" + 0.000*"muleteer" + 0.000*"ola" + 0.000*"or" + 0.000*"ordination" + 0.000*"peon" + 0.000*"mcmxiii"'),
 (2,
  '0.049*"hester" + 0.049*"alice" + 0.017*"austin" + 0.016*"nora" + 0.011*"kirk" + 0.010*"garth" + 0.003*"featherstone" + 0.002*"diagnose" + 0.000*"lillian" + 0.000*"trevlyn"'),
 (3,
  '0.085*"harry" + 0.074*"max" + 0.040*"baxter" + 0.018*"cleek" + 0.016*"narkom" + 0.012*"bradys" + 0.011*"superintendent" + 0.010*"carlyle" + 0.008*"king_brady" + 0.006*"brady"'),
 (4,
  '0.085*"florence" + 0.056*"jean" + 0.049*"holme" + 0.042*"barne" + 0.033*"watson" + 0.018*"doyle" + 0.012*"ronald" + 0.009*"moor" + 0.007*"hilda" + 0.007*"euston"'),
 (5,
  '0.006*"collin" + 0.003*"quid" + 0.002*"hamilton_cleek" + 0.002*"smather" + 0.002*"vanishing

start with the full text
overlap means reduce topics
expand your stop word list iteratively
Just keep taking out words that don't convey meaning

In [19]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.089*"mary" + 0.070*"leslie" + 0.038*"cynthia" + 0.035*"jeanne" + '
  '0.008*"delia" + 0.005*"naylor" + 0.002*"beaumaroy" + 0.002*"saffron" + '
  '0.002*"inkston" + 0.002*"alec"'),
 (1,
  '0.000*"portliness" + 0.000*"ranchero" + 0.000*"mina" + 0.000*"mozo" + '
  '0.000*"muleteer" + 0.000*"ola" + 0.000*"or" + 0.000*"ordination" + '
  '0.000*"peon" + 0.000*"mcmxiii"'),
 (2,
  '0.049*"hester" + 0.049*"alice" + 0.017*"austin" + 0.016*"nora" + '
  '0.011*"kirk" + 0.010*"garth" + 0.003*"featherstone" + 0.002*"diagnose" + '
  '0.000*"lillian" + 0.000*"trevlyn"'),
 (3,
  '0.085*"harry" + 0.074*"max" + 0.040*"baxter" + 0.018*"cleek" + '
  '0.016*"narkom" + 0.012*"bradys" + 0.011*"superintendent" + 0.010*"carlyle" '
  '+ 0.008*"king_brady" + 0.006*"brady"'),
 (4,
  '0.085*"florence" + 0.056*"jean" + 0.049*"holme" + 0.042*"barne" + '
  '0.033*"watson" + 0.018*"doyle" + 0.012*"ronald" + 0.009*"moor" + '
  '0.007*"hilda" + 0.007*"euston"'),
 (5,
  '0.006*"collin" + 0.003*"quid" + 0.002*"ha

In [20]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.883119396818502

Coherence Score:  0.3729110026788441


In [25]:
# Visualize the topics
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis = gensimvis.prepare(lda_model, corpus, id2word, mds = 'mmds')
#vis = gensimvis.prepare(model, corpus, dictionary, mds = 'mmds', sort_topics=False)
pyLDAvis.save_html(vis, 'lda02.html')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.371305 -0.106144       1        1  27.875614
12    -0.416590 -0.017821       2        1  26.069098
6     -0.386353 -0.191268       3        1  21.809278
17    -0.319202 -0.286273       4        1  17.185313
19    -0.427789  0.160550       5        1   6.677817
4      0.129044  0.189961       6        1   0.124936
3      0.069296  0.132536       7        1   0.093387
0      0.142760 -0.080593       8        1   0.042994
18     0.159573 -0.027267       9        1   0.039432
2      0.175581  0.022768      10        1   0.031343
7      0.133346  0.015054      11        1   0.013499
16     0.126993  0.028178      12        1   0.010444
8      0.114734  0.017857      13        1   0.009327
5      0.127110  0.022403      14        1   0.008099
13     0.122299  0.024977      15        1   0.005293
15     0.123096  0.018634      16        1   0.002842
14     0.124352  0.019112      17        1   0.000327
1      0.124352  0.019112      18        1   0.000321
9      0.124352  0.019112      19        1   0.000321
11     0.124352  0.019112      20        1   0.000316, topic_info=           Term         Freq         Total Category  logprob  loglift
1671        boy   5507.00000   5507.000000  Default   30.000  30.0000
337        girl   6496.00000   6496.000000  Default   29.000  29.0000
335         get   9711.00000   9711.000000  Default   28.000  28.0000
342          go  14848.00000  14848.000000  Default   27.000  27.0000
1734  detective   1515.00000   1515.000000  Default   26.000  26.0000
...         ...          ...           ...      ...      ...      ...
25         alec      0.00013    105.955772  Topic20  -10.702  -0.9500
26        alecs      0.00013      0.387206  Topic20  -10.702   4.6619
27       almost      0.00013   2165.991499  Topic20  -10.702  -3.9676
28        alone      0.00013   1220.263190  Topic20  -10.702  -3.3938
29         also      0.00013   1426.331372  Topic20  -10.702  -3.5498

[1133 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
0          1  0.214873  absolute
0          2  0.659438  absolute
0          3  0.007409  absolute
0          4  0.111141  absolute
1          1  0.580795    accent
...      ...       ...       ...
913        2  0.367538     young
913        3  0.234217     young
913        4  0.049209     young
913        5  0.052451     young
11251      7  0.811075        ze

[1281 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 13, 7, 18, 20, 5, 4, 1, 19, 3, 8, 17, 9, 6, 14, 16, 15, 2, 10, 12])

We preprocess the novels by lemmatizing the words, removing
punctuation, function words and names, and splitting the remaining text in chunks of 1000
tokens. We use MALLET to create a topic model with 50 topics. Fig. 1 shows an overview of the
topics with their proportion across the corpus.

Next steps: 

identify and remove named entities before tokenizing

use entire texts instead of first x characters

break into chunks of 1000 tokens for topic modeling -- how does this actually work if you are processing multiple books/texts?

iterate through models with various parameters